# Vicuna 4 bit quantized

In [1]:
import sys
import os
sys.path.append("../src")
import llm_utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

classes = ["War/Terror", "Conspiracy Theory", "Education", "Election Campaign", "Environment", 
              "Government/Public", "Health", "Immigration/Integration", 
              "Justice/Crime", "Labor/Employment", 
              "Macroeconomics/Economic Regulation", "Media/Journalism", "Religion", "Science/Technology"]

# Without context and classification only
## Example:
    ### Human: Classify the Tweet based on if it's about {label}. Use 1 or 0 as class.\n\nTweet: {tweet_text}\n### Assistant:\nClass = 
without_context_classification_only_df = pd.read_csv("../data/vicuna_4bit/generic_prompt_without_context_only_classification/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_first_character")
prediction_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(without_context_classification_only_df, classes, extraction_function)
without_context_classification_only = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

# without context and elaboration first
## Example:
    ### Human: Elaborate on whether you think the Tweet is about {label} or something else.\nTweet: {tweet_text}\n### Assistant:\nElaboration: 
    #Followup :\n Assign class 1 for {label} or 0 for not. \n###Assistant:\nClass:  
without_context_elaboration_first_df = pd.read_csv("../data/vicuna_4bit/generic_prompt_without_context_elaboration_first/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_last_float")
prediction_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(without_context_elaboration_first_df, classes, extraction_function)
without_context_elaboration_first = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

#with rules as context and classification only
## Example:
    ### Human: Based on rules, classify the Tweet based on if it's about {label}. Use 1 or 0 as class.\n\nRules: {rules}\nTweet: {tweet_text}\n### Assistant:\nClass:
with_rules_classification_only_df = pd.read_csv("../data/vicuna_4bit/generic_prompt_with_rules_only_classification/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_second_character")
prediction_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(with_rules_classification_only_df, classes, extraction_function)
with_rules_classification_only = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

# with rules as context and elaboration first
## Example:
    ### Human: Based on rules, elaborate whether you think the Tweet is about {label}.\nRules: {rules}\nTweet: {tweet_text}\n### Assistant:\nElaboration: 
    #Followup :\n Assign class 1 for {label} or 0 for not. \n###Assistant:\nClass: 
with_rules_elaboration_first_df = pd.read_csv("../data/vicuna_4bit/generic_prompt_with_rules_elaboration_first/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_first_character")
prediction_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(with_rules_elaboration_first_df, classes, extraction_function)
with_rules_elaboration_first = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

TypeError: 'NoneType' object is not callable

In [3]:
import sys
import os
sys.path.append("../src")
import llm_utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

classes = ["War/Terror", "Conspiracy Theory", "Education", "Election Campaign", "Environment", 
              "Government/Public", "Health", "Immigration/Integration", 
              "Justice/Crime", "Labor/Employment", 
              "Macroeconomics/Economic Regulation", "Media/Journalism", "Religion", "Science/Technology"]


text_davinci_003_turbo_without_context_elaboration_first_v02_df = pd.read_csv("../data/openai_text_davinci_003/generic_prompt_without_context_elaboration_first_v03/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_nth_character", 1, False)
text_davinci_003_turbo_without_context_elaboration_first_v02_predictions_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(text_davinci_003_turbo_without_context_elaboration_first_v02_df, classes, extraction_function)
text_davinci_003_turbo_without_context_elaboration_first_v02 = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

llm_utils.print_confusion_matrix(text_davinci_003_turbo_without_context_elaboration_first_v02)

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4995    NaN
4996    NaN
4997    NaN
4998    NaN
4999    NaN
Name: War/Terror_pred, Length: 5000, dtype: object
War/Terror                        id campaign_name                                               text                                        annotations                                   normalized_tweet War/Terror_pred Conspiracy Theory_pred  ... Immigration/Integration_pred Justice/Crime_pred Labor/Employment_pred Macroeconomics/Economic Regulation_pred Media/Journalism_pred Religion_pred Science/Technology_pred
96    1155203013805559809    GRU_202012  In the city of Ras al-Ain #Hasaka and the surr...                                     ['War/Terror']  In the city of Ras al-Ain #Hasaka and the surr...            None                    NaN  ...                          NaN                NaN                   NaN                                     NaN                   NaN           NaN                 

/home/bruno/anaconda3/envs/my_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bruno/anaconda3/envs/my_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bruno/anaconda3/envs/my_env/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/bruno/anaconda3/envs/my_env/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)

In [2]:
from tabulate import tabulate

models = [
    {
        "name": "No context - Classification only",
        "data": without_context_classification_only,
    },
    {
        "name": "No context - Elaboration first",
        "data": without_context_elaboration_first,
    },
    {
        "name": "With rules - Classification only",
        "data": with_rules_classification_only,
    },
    {
        "name": "With rules - Elaboration first",
        "data": with_rules_elaboration_first,
    }

]

for model in models:
    llm_utils.print_confusion_matrix(model["data"])

Confusion matrix for War/Terror:
[[56  9]
 [17 48]]

Confusion matrix for Conspiracy Theory:
[[54 11]
 [26 39]]

Confusion matrix for Education:
[[43 22]
 [ 9 56]]

Confusion matrix for Election Campaign:
[[42 23]
 [17 48]]

Confusion matrix for Environment:
[[36 29]
 [12 53]]

Confusion matrix for Government/Public:
[[32 33]
 [23 42]]

Confusion matrix for Health:
[[45 20]
 [16 49]]

Confusion matrix for Immigration/Integration:
[[52 13]
 [16 49]]

Confusion matrix for Justice/Crime:
[[28 37]
 [ 8 57]]

Confusion matrix for Labor/Employment:
[[47 18]
 [18 47]]

Confusion matrix for Macroeconomics/Economic Regulation:
[[63  2]
 [48 17]]

Confusion matrix for Media/Journalism:
[[31 94]
 [ 1  4]]

Confusion matrix for Religion:
[[97 31]
 [ 0  2]]

Confusion matrix for Science/Technology:
[]

Confusion matrix for War/Terror:
[[38 25]
 [ 6 59]]

Confusion matrix for Conspiracy Theory:
[[45 20]
 [26 38]]

Confusion matrix for Education:
[[24 41]
 [ 2 63]]

Confusion matrix for Election Camp

In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 32.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 66.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.3 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=55c8fa74348198d4344fe896c217ac8ef74d5fe65a57d200f2c5438bc8c94f12
  Stored in directory: /home/bruno/.var/app/com.visualstudio.code/cache/pip/wheels/82/35/dc/f88ec71edf2a5596bd72a8fa1b697277e0fcd3cde83048

In [9]:
!pip install streamlit-components

ERROR: Could not find a version that satisfies the requirement streamlit-components (from versions: none)
ERROR: No matching distribution found for streamlit-components


In [6]:
import streamlit as st
import pandas as pd

# Title
st.title("Classification Report and Confusion Matrix Viewer")

# Model selection
model_names = [model["name"] for model in models]
selected_model = st.sidebar.selectbox("Select model", model_names)

# Class selection
class_options = classes
selected_class = st.sidebar.selectbox("Select class", class_options)

# Get the selected data
selected_model_data = None
for model in models:
    if model["name"] == selected_model:
        selected_model_data = model["data"]
        break

selected_confusion_matrix = selected_model_data["confusion_matrices"][selected_class]
selected_classification_report = selected_model_data["classification_reports"][selected_class]

# Display the confusion matrix and classification report
st.header("Confusion Matrix")
st.write(selected_confusion_matrix)

st.header("Classification Report")
st.write(pd.DataFrame(selected_classification_report).transpose())

In [7]:
!streamlit run report_viewer.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: report_viewer.py


In [5]:
without_contex_only_classification_df[without_contex_only_classification_df["Macroeconomics/Economic Regulation_pred"].notna()].copy().annotations.to_list()[0:10]

["['War/Terror']",
 "['Others']",
 "['Macroeconomics/Economic Regulation']",
 "['Government/Public', 'Macroeconomics/Economic Regulation']",
 "['Government/Public', 'Macroeconomics/Economic Regulation', 'Media/Journalism', 'War/Terror']",
 "['Government/Public', 'Macroeconomics/Economic Regulation']",
 "['Others']",
 "['Government/Public', 'Macroeconomics/Economic Regulation']",
 "['Education', 'Media/Journalism']",
 "['Justice/Crime']"]

In [4]:
without_contex_only_classification_df[without_contex_only_classification_df["Macroeconomics/Economic Regulation_pred"].notna()].copy().text.to_list()[0:10]

['RT @Ben_Jonson_1985: #Feylak Ar-Rahman militants located in #EasternGhouta do not release #civilians from the fighting zone. The use of a l…',
 'Women’s March Organizer Wants to “Take Vagina Away” from Female Genital Mutilation Survivor https://t.co/MmBm9WD6Ux https://t.co/T9rBUrOySf',
 'IMF has demanded to take credit apartments of Ukrainians\nhttp://t.co/2hcqYxfQHc #Ukraine',
 'Libertarianism is nothing but liberty for exploiters, racists, oppressors and parasites. Socialism is liberty for the people; the working class. https://t.co/uyuugJzTpq',
 '🇮🇱🇵🇸Israel is now experiencing a moment of heightened political, economic, and military power is no excuse to avoid dealing with the Palestinian issue, writes #valdaiclub expert Amos Yadlin.\n\nhttps://t.co/7IPavmsHJh',
 'Following the power cut crisis faced by Zimbabwe where electricity is only available between 10pm and 6am, people have adjusted their daily routines to make the most out of the electricity when it is available.\n\nRead f